<a href="https://colab.research.google.com/github/shawnmuhr/BIOL_398/blob/main/HW_Solutions/biol300_hw2_solns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 import random

# Exercise 0: Practicing using GitHub (10 points)

We will be checking for commit messages, and to practice writing them, they will be worth points on this first assignment. To earn full points, *each* team member must submit *at least one* commit message during the process of working on this homework.



# Exercise 1a: Melting temperature calculator (10 pts)

Let's say you want to make your own function for validating the design of a given PCR primer sequence. (Note many tools already exist [online](https://bioinfo.ut.ee/primer3-0.4.0/), but it's so much more satisfying to write your own!) As the first step, we want to calculate the melting temperature of a given primer sequence. We will use the following equation:

$$ T_m = 81.5 + 0.41 \times (\%GC) - 675/N,$$

where $T_m$ is the melting temperature in $^\circ$C, %GC is the GC content (**in percent, not decimal!**), and $N$ is the length of the sequence. Write a function that will take in a DNA sequence, and output the melting temperature.

Note you may want to make use of your `GC_content()` function from last week. You can redefine that function here to call it within the `melting_temp()` function below. 

# Solution 1a:

First, I'll re-define the `GC_content` function that I wrote last time:

In [ ]:
def GC_content(DNA_seq):
  """Computes the GC content (as a float) given the provided DNA sequence 
  (a string)"""

  # count Cs and Gs, respectively
  C_count = DNA_seq.count('C')
  G_count = DNA_seq.count('G')

  # comupte GC content as a fraction 
  GC_frac = (C_count + G_count) / len(DNA_seq)

  return GC_frac

Now I can write the melting temperature function, calling my `GC_content` function from above.

In [ ]:
def melting_temp(DNA_seq):
  """computes the melting temperature of provide DNA sequence""" 

  N = len(DNA_seq)

  percent_GC_content = GC_content(DNA_seq)*100

  T_m = 81.5 + 0.41*percent_GC_content - 675/N

  return T_m

A few tests are included below, with the expected value in the comment above the function call.

In [ ]:
# should be 71.8 (with rounding)
melting_temp("ATCGATCAGTTACGATAGCCGAC") 

71.76086956521738

In [ ]:
# should be 64.5
melting_temp("GCATCGATCGATTACGAC") 

64.5

# Exercise 1b: Primer checker (20 pts)

Let's now develop our full primer checker. For this, we will make reference to [ThermoFisher's guidlines for primer design](https://www.thermofisher.com/blog/behindthebench/pcr-primer-design-tips/). Specifically, your function should to make the following checks on a given DNA sequence:

- GC content to be between 40 and 60%. (Again you can use your function from last week.)
- Primer ends in a G or C
- Length between 18 and 30 bases
- Make sure the melting temperature is between 65$^\circ$C and 75$^\circ$C
- Avoid strings of 4 of the same nucleotide (i.e. 'AAAA', 'CCCC', etc.)

You'll want to check each of these rules in turn and `print` output to let the user know if they have violated any (or all!) of these design guidelines. If everything checks out, you can return the melting temperature so you can go about designing your PCR reaction!

Note there are *a lot* of checks to do here. To keep the `primer_checker()` function more manageable, you may want to write some other smaller functions that you can call in this main function. How you approach the code is up to you.  

# Solution 1b:

Below I structure my `primer_checker` function as a set of consecutive `if` statements. This is probably reaching the max of what I would want to tackle with a single function, so it would also have been reasonable to break it out into several smaller functions.

In [ ]:
def primer_checker(DNA_seq):
  """Provides output (in the form of print statements) to the user,
  letting them know if the primer sequence has any design flaw.
  Otherwise, returns the melting temperature"""

  # primer is considered good, unless we fail one of the tests below
  good_primer = True

  # check for length
  if len(DNA_seq) < 18:
    good_primer = False
    print("Your primer is too short")
  if len(DNA_seq) > 30:
    good_primer = False
    print("Your primer is too long")

  # check for ending in G or C
  if DNA_seq[-1] not in ["G","C"]:
    good_primer = False
    print("Your primer does not end in a G or C")

  # check for GC content
  if GC_content(DNA_seq) < 0.4:
    good_primer = False
    print("Your primer's GC content is too low")
  if GC_content(DNA_seq) > 0.6:
    good_primer = False
    print("Your primer's GC content is too high")

  # check for melting temperature
  T_m = melting_temp(DNA_seq)
  if T_m < 65:
    good_primer = False
    print("Your primer's melting temperature is too low") 
  if T_m > 75:
    good_primer = False
    print("Your primer's melting temperature is too high")

  # check for repeats of four nucleotides in a row
  bad_seqs = ["AAAA","CCCC","GGGG","TTTT"]
  for i in range(len(DNA_seq)):
    if DNA_seq[i:i+4] in bad_seqs:
      good_primer = False
      print("Your primer has a nucleotide repeated at least 4 times in a row")
      break # so this print statment only appears once

  # return the melting temperature if everything checks out!
  if good_primer:
    return melting_temp(DNA_seq)
  
  # otherwise return nothing
  return

# Exercise 1c: Testing your primer checker (10 points)

For each of the five rules we put in place for good primer design, test out your function below to demonstrate that it responds appropriately for when the rule is followed and when the rule is broken. 

# Soultion 1c:

First I'll start with a seqeunce that passes all the tests, and then test the various checks in turn:


In [ ]:
primer_checker("ATCGACTAAGCTAGCATCGATGCG")

73.875

Let's test making the sequence too short and too long:

In [ ]:
primer_checker("ATCGACTAAGCTAC")
print()
primer_checker("ATCGACTAAGCTAGCATCGATGCGATCGACCAAG")

Your primer is too short
Your primer's melting temperature is too low

Your primer is too long
Your primer's melting temperature is too high


Great. That worked. We also got melting temperature warnings, but that makes sense as we change the number of basepairs. Next let's test ending the primer without a C or G:

In [ ]:
primer_checker("ATCGACTAAGCTAGCATCGATGCGA") 
print()
primer_checker("ATCGACTAAGCTAGCATCGATGCGT") 

Your primer does not end in a G or C

Your primer does not end in a G or C


Next let's alter the GC content and see how our function responds:

In [ ]:
primer_checker("ATTAACTAAGATAGTATCGATACG") # AT-rich
print()
primer_checker("AGCGACCAGGCTAGCAGCGAGGCG") # GC-righ

Your primer's GC content is too low

Your primer's GC content is too high
Your primer's melting temperature is too high


We already saw that the melting temperature checks worked above, so let's lastly check for string of nucleotides repeating four times or more. 

In [ ]:
primer_checker("ATCGACTAAAAGCTAGCATCGATGCG") # A repeated
print()
primer_checker("ATCGACTTTTAAGCTAGCATCGATGCG") # T repeated
print()
primer_checker("ATCGACTAAGCTAGCATCGAGGGG") # G repeated
print()
primer_checker("ATCGACTAACCCCGCATCGAGGGG") # C repeated

Your primer has a nucleotide repeated at least 4 times in a row

Your primer has a nucleotide repeated at least 4 times in a row

Your primer's melting temperature is too high
Your primer has a nucleotide repeated at least 4 times in a row

Your primer's GC content is too high
Your primer's melting temperature is too high
Your primer has a nucleotide repeated at least 4 times in a row


We've now seen all the possible warning messages, so it seems our function works well!

# Exercise 2: Mutate sequence (20 points)

Last time we wrote code to mutate a single nucleotide to another nucleotide. Now we will expand the functionality to mutagenize an entire DNA sequence at a specified mutation rate. For example, if we have a DNA sequence of length ten and mutagenize at rate 0.1, we would expect *on average* one nucleotide to get mutated, but it may very well happen that more than one (or no) nucleotides get mutated. You may want to reference [Python's documentation about the random module](https://docs.python.org/3/library/random.html) for functions that may help you here.

Again, I will reuse the functions from last time, so I copy the `mutate_nt` function here.

In [ ]:
def mutate_nt(nt):
  """Returns a randomly selected mutated nucleotide from the inputted 
  nucleotide"""

  # initialize all possible options, then exclude the nt provided
  options = ["A","T","C","G"]
  options.remove(nt)

  # return one of the remaining options at random
  return random.choice(options)

In [ ]:
import random

In [ ]:
def mutate_seq(DNA_seq, mut_rate):
  """Returns a mutated DNA sequence at the mutation rate provided"""

  # convert DNA to a list, so as to be more easily edited
  DNA_list = list(DNA_seq)

  # loop through DNA
  for i in range(len(DNA_list)):

    # get a random number from 0 to 1
    rand_num = random.random()

    # using that random number, decide whether to mutate or not
    if rand_num < mut_rate:
      rand_nuc = mutate_nt(DNA_list[i])
      DNA_list[i] = rand_nuc

  # combine the list back into a string for returning
  DNA_seq = "".join(DNA_list)

  return DNA_seq

Make sure to try your code out on a few sequence to see that they do in fact mutate! 

Just to see the mutations easily, I use a loop to mutate the same sequence 10 separate times.

In [ ]:
for i in range(10):
  print(mutate_seq("AAATTTCCCGGG", 0.2))

AAATTTTTCAGG
AAAAGTCCCGGG
AAATTTCCCAGG
AGGTTTCGCGGG
ATACTTCTCGGG
AAATTTACCTGG
AAATTTCCCGGA
AAATTTCCCGGA
AAATTTCACGGT
GAGTTTCCCAGG


This demonsrates the randomness I was hoping for. Sometimes there's one or two mutations, and sometimes there's no mutations! It's all just a matter of chance.

# Exercise 3: ORF finder (30 pts)

Again building off the code we wrote last time, we will now write an Open Reading Frame (ORF) finder. This means rather than just blinding translating from the start of the provided DNA sequence, we will want to be mindful of start codons and various reading frames. Your function should return a `list` of possible ORFs translated into amino acid sequences, considering all possible start codons in all possible reading frames. 

As an example, your code may output something like:
`["MRPSLRAMHGAVRT*","MHGAVRT*","MAETMLP","MLP"]`, where some ORFs can be a subset of another ORFs, and some ORFs don't have a terminating stop codon because they've reached the end of the provided sequence.

Again, we need to define the dictionary for corresponding codons to amino acids. 

In [ ]:
aa_dict = \
{'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L', 'TCT': 'S', 'TCC': 'S', 
 'TCA': 'S', 'TCG': 'S', 'TAT': 'Y', 'TAC': 'Y', 'TAA': '*', 'TAG': '*', 
 'TGT': 'C', 'TGC': 'C', 'TGA': '*', 'TGG': 'W', 'CTT': 'L', 'CTC': 'L', 
 'CTA': 'L', 'CTG': 'L', 'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P', 
 'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q', 'CGT': 'R', 'CGC': 'R', 
 'CGA': 'R', 'CGG': 'R', 'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M', 
 'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T', 'AAT': 'N', 'AAC': 'N', 
 'AAA': 'K', 'AAG': 'K', 'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R', 
 'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V', 'GCT': 'A', 'GCC': 'A', 
 'GCA': 'A', 'GCG': 'A', 'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E', 
 'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'}

And now you can write your function.

Using the `translate` function from last time, I can call it in my `ORF_finder` function.

In [ ]:
def translate(DNA_seq):
  """Returns a string of the amino acid sequence that results from translating
  the inputted DNA sequence"""

  # round down to get number of codons
  num_codons = int(len(DNA_seq)/3)

  # initilize string of amino acids
  aa_seq = ""
  
  for i in range(num_codons):

    # determine codon and corresponding amino acid
    codon = DNA_seq[i*3:i*3+3]
    aa = aa_dict[codon]

    # update amino acid string with new amino acid
    aa_seq = aa_seq + aa

    # check for stop codon
    if aa == "*":
      return aa_seq

  return aa_seq

In [ ]:
def ORF_finder(DNA_seq):
  """Returns a list of all translated ORFs in the provided
  DNA sequence"""

  # initilize list to save ORF
  ORFs = []

  # loop through DNA and look for start codons
  for i in range(len(DNA_seq)):
    if DNA_seq[i:i+3] == "ATG":

      # once you find a start codon,
      # call the translate function to get the ORF from this start codon
      ORF = translate(DNA_seq[i:])

      # add ORF to list
      ORFs.append(ORF)
  
  return ORFs

Test out your function on a few inputs to make sure it works as expected. A few things you may want to check for:
- Does your function find multiple ORFs?
- Does your function find overlapping ORFs?
- Does your function find ORFs in different reading frames?



Below I test it out, seeing that it does find ORFs that are subsets of each other (e.g. the first and second), as well as an ORF in a different reading frame (e.g. the third one).

In [ ]:
ORF_finder("ATGCAGCATCAGATGCATCGACAATGCGACGACAGTCAGCATAGACGCA")

['MQHQMHRQCDDSQHRR', 'MHRQCDDSQHRR', 'MRRQSA*']

# How long did this take? 

With a new course and new assignments, I want to be conscientious with how much time this course takes. Please let me know how long this took, so I can adjust future homeworks if needed.

# References

If you referenced any external sources for completing this homework, please list them below. (Just the links are fine.)

# Submitting your homework

Please make sure to state what each group member contribute and have each group member "sign off" that they agree they are satisfied with the final submission of this homework.

You will submit this homework (and all subsequent homeworks via GitHub). Unless you have an approved extension or opt to submit the homework late (with a 10% deduction per day), your homework will be graded based on what is submitted on GitHub at the time of the deadline. So don't forget to push! 